In [1]:
import pandas as pd
import requests
import json

In [2]:
def execute_req(api_url, authorisation, parameters):
    """
    Execute request of data
    """
    r = requests.get(
        api_url,
        params=parameters,
        auth=(authorisation, '')
    )
    if r.status_code != 200:  # see HTTP errors
        print("HTTP ERROR")
        return
    else:
        result = json.loads(r.content.decode())
        return result

def page_gares(numero_page):
    return requests.get(
        ('https://api.sncf.com/v1/coverage/sncf/stop_areas?start_page={}').format(numero_page),
        auth=("42f6483b-fd9a-483a-89b8-4286395fd523", ''))

In [3]:
page_initiale = page_gares(0)
item_per_page = page_initiale.json()['pagination']['items_per_page']
total_items = page_initiale.json()['pagination']['total_result']
dfs = []

In [4]:
page = 1
stations_page = page_gares(page)
ensemble_stations = stations_page.json()

In [5]:
stations = ensemble_stations['stop_areas']

In [6]:
stations[1]['codes'][1]['value']

'87781278'

In [7]:
stations_2 = pd.DataFrame(stations)

In [8]:
def get_insee(adm_dict):
    if isinstance(adm_dict, float):
        return -1
    else:
        adm_dict = adm_dict[0]
        if 'insee' in adm_dict.keys():
            return adm_dict['insee']
        else:
            return -1

def get_lat(adm_dict):
    if isinstance(adm_dict, float):
        return -1
    else:
        adm_dict = adm_dict[0]
        return adm_dict['coord']['lat']

def get_lon(adm_dict):
    if isinstance(adm_dict, float):
        return -1
    else:
        adm_dict = adm_dict[0]
        return adm_dict['coord']['lon']

In [28]:
stations_2['insee'] = stations_2['administrative_regions'].apply(get_insee)

In [29]:
stations_2.head(5)

,codes,name,links,coord,label,administrative_regions,timezone,id,insee
0,"[{'type': 'source', 'value': '87757559'}, {'ty...",Agay,[],"{'lat': '43.43137', 'lon': '6.8565'}",Agay (Saint-Raphaël),"[{'insee': '83118', 'name': 'Saint-Raphaël', '...",Europe/Paris,stop_area:SNCF:87757559,83118
1,"[{'type': 'source', 'value': '87781278'}, {'ty...",Agde,[],"{'lat': '43.31728', 'lon': '3.466203'}",Agde (Agde),"[{'insee': '34003', 'name': 'Agde', 'level': 8...",Europe/Paris,stop_area:SNCF:87781278,34003
2,"[{'type': 'source', 'value': '87586008'}, {'ty...",Agen,[],"{'lat': '44.207967', 'lon': '0.6208669999999999'}",Agen (Agen),"[{'insee': '47001', 'name': 'Agen', 'level': 8...",Europe/Paris,stop_area:SNCF:87586008,47001
3,"[{'type': 'source', 'value': '87595157'}, {'ty...",Agonac,[],"{'lat': '45.286605', 'lon': '0.748653'}",Agonac (Agonac),"[{'insee': '24002', 'name': 'Agonac', 'level':...",Europe/Paris,stop_area:SNCF:87595157,24002
4,"[{'type': 'source', 'value': '87783399'}, {'ty...",Aguessac,[],"{'lat': '44.153724', 'lon': '3.097892'}",Aguessac (Aguessac),"[{'insee': '12002', 'name': 'Aguessac', 'level...",Europe/Paris,stop_area:SNCF:87783399,12002


In [9]:
for page in range(int(total_items / item_per_page) + 1):
    stations_page = page_gares(page)

    ensemble_stations = stations_page.json()

    if 'stop_areas' not in ensemble_stations:
        # pas d'arrêt
        continue

    stations = ensemble_stations['stop_areas']
    stations = pd.DataFrame(stations)
    stations['insee'] = stations['administrative_regions'].apply(get_insee)
    # stations['uic'] = stations['codes'].apply(lambda x: x[1]['value'])
    stations['lat'] = stations['coord'].apply(lambda x: x['lat'])
    stations['lon'] = stations['coord'].apply(lambda x: x['lon'])
    stations['lat_city'] = stations.administrative_regions.apply(get_lat)
    stations['lon_city'] = stations.administrative_regions.apply(get_lon)

    dfs.append(stations)
    if page % 10 == 0:
        print("Page", page, "---", stations.shape)


Page 0 --- (25, 13)
Page 10 --- (25, 13)
Page 20 --- (25, 13)
Page 30 --- (25, 13)
Page 40 --- (25, 13)
Page 50 --- (25, 13)
Page 60 --- (25, 13)
Page 70 --- (25, 13)
Page 80 --- (25, 13)
Page 90 --- (25, 13)
Page 100 --- (25, 13)
Page 110 --- (25, 13)
Page 120 --- (25, 13)
Page 130 --- (25, 13)
Page 140 --- (25, 13)
Page 150 --- (25, 13)
Page 160 --- (25, 13)
Page 170 --- (25, 13)
Page 180 --- (25, 13)
Page 190 --- (25, 13)


In [8]:
gares = pd.concat(dfs)
gares.to_csv("./data/ensemble_gares.csv")

In [10]:
def get_name(adm_dict):
    if isinstance(adm_dict, float):
        return -1
    else:
        adm_dict = adm_dict[0]
        if 'name' in adm_dict.keys():
            return adm_dict['name']
        else:
            return -1

In [11]:
gares['city'] = gares['administrative_regions'].apply(get_name)

In [12]:
gares.to_csv("./data/ensemble_gares.csv")

In [1]:
import pickle

In [2]:
with (open('results_paris.pickle', "rb")) as openfile:
    while True:
        try:
            objects = pickle.load(openfile)
        except EOFError:
            break

In [3]:
result = objects
total_time = {}
for city in result.keys():
    total_time[city] = [min(result[city]['total_time']), min(result[city]['duration'])]